 # Сопоставление геоназваний с унифицированными именами

## Содержание

**[1. Обзор данных](#1)**

- **[1.1 Ввод функций и констант](#1.1)**
- **[1.2 Подготовка к загрузке данных в SQL](#1.2)**


**[2. Исследование данных](#2)**

- **[2.1. Тестовый датсет](#2.1)**
- **[2.2. Датасет с различными гео-показателями о всех городах с населением 15 000+ или столицах](#2.2)**
- **[2.3. Названия административных подразделений на английском языке](#2.3)**
- **[2.4. Обучающий датасет с альтернативными именами и кодами языков](#2.4)**


**[3. Обучение](#3)**

- **[3.1. Евклидово расстояние для кириллицы](#3.1)**
- **[3.2. Косинусное расстояние](#3.2)**
- **[3.3. Евклидово расстояние с переводом в латиницу](#3.3)**
- **[3.4. Расстояние Левенштейна для русского текста](#3.4)**
- **[3.5. Расстояние Левенштейна с переводом в латиницу](#3.5)**

**[4. Проверка точности модели](#4)**

**[5. Формирование словаря ответов](#5)**


**Заказчик.** Карьерный центр Яндекс Практикум

**Цель.** Сопоставление произвольных гео-названий с унифицированными именами geonames для внутреннего использования Карьерным центром

**Цель исследования.** Разработка модели, которая достигнет наилучшего значения по метрике классификации `accuracy`.

**Задачи:**
- загрузить и ознакомится с данными,
- провести предварительную обработку,
- сделать полноценный разведочный анализ,
- создать решение для подбора наиболее подходящих названий с geonames. Например, `Ереван -> Yerevan`,
- разработку решения вести на примере РФ и стран наиболее популярных для релокации: `Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия`. Города с населением от `15000 человек`,
- предусмотреть возможность масштабирования разработанного решения на сервере заказчика,
- обеспечить возврат полей `geonameid, name, region, country, cosine similarity`. Обеспечить формат данных на выходе: список словарей. Например `[{dict_1}, {dict_2}, …. {dict_n}]` где словарь - одна запись с указанными полями

**Входные данные.**

Четыре файла в формате `.csv` и `.txt` с данными о гео-названиях:

1. `geo_test.csv` – тестовый датасет (345 строк).
2. `admin1CodesASCII.tx` – датасет с названиями административных подразделений на английском языке (~ 3 800)
3. `alternateNamesV2.txt` – обучающий датасет с альтернативными именами и кодами языков (~ 16 000 000)
4. `cities15000.txt` – датасет с различными гео-показателями о всех городах с населением 15 000+ или столицах (~ 27 000)

**Формальная задача.** Определить правильное название территории и преобразовать его в формат имени geonames

**Ожидаемый результат.** Построена модель для определения имени в формате geonamesс наилучшим значением метрики классификации `accuracy`.


## Обзор данных <a id='1'></a>

**Название файла 1**

`geo_test.csv`

**Признаки**

`query` – запрос название географического места

**Целевые признаки**

- `name` – имя географического места
- `region` – регион / область географического места
- `country` – страна географического места


**Название файла 2**

`admin1CodesASCII.txt`

- `code` – код названия административного подразделения
- `name` - название географической точки на родном языке
- `name ascii` - название географической точки на английском языке
- `geonameid` - идентификатор записи в базе данных геоназваний


**Название файла 3**

`alternateNamesV2.txt`

- `alternateNameId` - идентификатор альтернативного имени
- `geonameid` - идентификатор записи в базе данных геоназваний
- `isolanguage`  - код языка ISO 639
- `alternate name` - альтернативное имя или вариант имени
- `isPreferredName` - '1', если это альтернативное имя является официальным/предпочтительным именем.
- `isShortName` - «1», если это короткое имя, например «Калифорния» для «Штата Калифорния».
- `isColloquial`  - '1', если это альтернативное имя является разговорным или жаргонным термином. Пример - «Большое яблоко» вместо «Нью-Йорк».
- `isHistoric`  - '1', если это альтернативное имя является историческим и использовалось в прошлом. Пример «Бомбей» вместо «Мумбаи».
- `from`  - периода, с которого использовалось имя
- `to`  - период, до которого использовалось имя


**Название файла 4**

`cities15000.txt`

**Признаки**

- `alternatenames` - альтернативные имена, разделенные запятыми
- `latitude` - широта в десятичных градусах
- `longitude` - долгота в десятичных градусах
- `feature class` 
- `feature code` 
- `country code` - код страны
- `cc2` - альтернативные коды страны
- `admin1 code` - код первой административной единицы 
- `admin2 code` - код второй административной единицы
- `admin3 code` - код административной единицы третьего уровня
- `admin4 code` - код административной единицы четвертого уровня
- `population` – численность населения
- `elevation` – высота над уровнем моря
- `dem` - цифровая модель рельефа
- `timezone` - идентификатор часового пояса
- `modification date` - дата последней модификации

**Целевой признак**

- `geonameid` - идентификатор записи в базе данных геоназваний.
- `name` - название географической точки
- `asciiname` - название географической точки простыми символами


In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.engine.url import URL

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

from fuzzywuzzy import fuzz
from fuzzywuzzy import process # расстояние Левенштейна

from sentence_transformers import SentenceTransformer, util
import torch

from sklearn.metrics import accuracy_score


### Ввод функций и констант <a id='1.1'></a>

In [2]:
# Функция исследования датасета
def info_df(df_name):
    st = df_name.shape[0]
    col = df_name.shape[1]
    zerro = df_name.isna().sum()
    print(f'Размер таблиы: {col} столбцов и {st} строк')
    print(f'Общее количество пропусков - {zerro.sum()}')
    if len(zerro.loc[lambda x: x!=0]) != 0:
        print (zerro.loc[lambda x: x!=0])
    print()
    print(f'Количество явных дубликатов: {df_name.duplicated().sum()}')
    print()
    print(df_name.info())
    print()
    display(df_name.describe().round(2))

In [3]:
# Функция создания словаря из списка и ключа. Для преобразования вложенного списка в плоский
def alternatenames(text, geonameid):
    list_text = text.split(',')
    dict_text = {}
    for i in list_text:
        dict_text[i] = geonameid
    return dict_text
    

In [4]:
# Функция для очистки текста
def clear_text(text):
    text = text.lower()
    t_str = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9-]', " ", text)
    t_str = " ".join(t_str.split())
    return t_str

In [5]:
# Функция для транслитирования текста из кириллицы в латинницу
def translit(text):
    legend = {
        'а': 'a',
        'б': 'b',
        'в': 'v',
        'г': 'g',
        'д': 'd',
        'е': 'e',
        'ё': 'yo',
        'ж': 'zh',
        'з': 'z',
        'и': 'i',
        'й': 'y',
        'к': 'k',
        'л': 'l',
        'м': 'm',
        'н': 'n',
        'о': 'o',
        'п': 'p',
        'р': 'r',
        'с': 's',
        'т': 't',
        'у': 'u',
        'ф': 'f',
        'х': 'h',
        'ц': 'ts',
        'ч': 'ch',
        'ш': 'sh',
        'щ': 'shch',
        'ъ': 'y',
        'ы': 'y',
        'ь': "'",
        'э': 'e',
        'ю': 'yu',
        'я': 'ya',
        }
    new_text = ''
    text = text.lower()
    for i in text:
        if i in legend:
            new_text += legend[i]
        elif i == ' ':
            new_text += ' '
        else:
            new_text += i

    return new_text

In [6]:
# Расположение файлов
patch = 'C:\\Users\\User\\first_notebook\\Praktikum\\geonames\\data_base\\'
patch_1 = '' 

In [7]:
# Коды исследуемых стран
country_code = ['RU', 'BY', 'AM', 'KZ', 'KG', 'RS', 'TR']

# Названия исследуемых стран
country_name = {'RU': 'Russia', 
                'BY': 'Belarus', 
                'AM': 'Armenia', 
                'KZ': 'Kazakhstan', 
                'KG': 'Kyrgyzstan', 
                'RS': 'Serbia', 
                'TR': 'Turkey'}

# Пароль на серевер
password_sql = ''

### Подготовка к загрузке данных в SQL <a id='1.2'></a>

In [8]:
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres', 
    'password': password_sql, 
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
    'query': {}
}  

engine = create_engine(URL(**DATABASE))

## Исследование данных<a id='2'></a>

### Тестовый датсет <a id='2.1'></a>

In [9]:
# Чтение файла
try:
    df_test = pd.read_csv(patch + 'geo_test.csv', sep=';')
except FileNotFoundError:
    df_test = pd.read_csv(patch_1 + 'geo_test.csv', sep=';')

In [10]:
df_test.head(3)

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan


Оставим только исследуемые страны

In [11]:
df_test['country'].unique()

array(['Russia', 'Kyrgyzstan', 'Kazakhstan', 'Belarus', 'Serbia',
       'Armenia', 'Georgia'], dtype=object)

In [12]:
df_test = df_test.loc[df_test['country']!='Georgia'].copy()

In [13]:
# Исследуем датасет
info_df(df_test)

Размер таблиы: 4 столбцов и 340 строк
Общее количество пропусков - 0

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 0 to 344
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   query    340 non-null    object
 1   name     340 non-null    object
 2   region   340 non-null    object
 3   country  340 non-null    object
dtypes: object(4)
memory usage: 13.3+ KB
None



,query,name,region,country
count,340,340,340,340
unique,340,313,111,6
top,Смоленск,Saint Petersburg,Moscow Oblast,Russia
freq,1,3,49,291


### Датасет с различными гео-показателями о всех городах с населением 15 000+ или столицах<a id='2.2'></a>

In [14]:
# Названия столбцов
cities_names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude',
              'longitude', 'feature class','feature code', 'country code', 'cc2',
              'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population',
              'elevation', 'dem', 'timezone', 'modification date']

In [17]:
# Чтение файла
try:
    df_cities1500 = pd.read_csv(patch + 'cities15000.txt', sep='\t', 
                                lineterminator='\n', header=None, names=cities_names)
except FileNotFoundError:
    df_cities1500 = pd.read_csv(patch_1 + 'cities15000.txt', sep='\t', 
                                lineterminator='\n', header=None, names=cities_names)

In [18]:
# Создание таблицы на сервере
df_cities1500.to_sql('cities1500', con=engine)

932

In [19]:
query = """

SELECT * 
FROM cities1500 
LIMIT 5

"""

pd.read_sql_query(text(query), con=engine.connect())

,index,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,None,08,None,None,None,15853,None,1033,Europe/Andorra,2008-10-15
1,1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,None,07,None,None,None,20430,None,1037,Europe/Andorra,2020-03-03
2,2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,None,07,None,None,None,62747,None,2,Asia/Dubai,2019-10-24
3,3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,None,05,None,None,None,351943,None,2,Asia/Dubai,2019-09-09
4,4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,None,01,103,None,None,63482,None,118,Asia/Dubai,2019-10-24


In [20]:
# Выгрузка таблицы с сервера

query = """

SELECT * 
FROM cities1500 

"""

df_cities1500 = pd.read_sql_query(text(query), con=engine.connect())

In [21]:
# Оставим только исследуемые страны
df_cities1500 = df_cities1500.query('`country code` in @country_code').reset_index(drop=True)

In [22]:
df_cities1500.head(3)

,index,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,94,174875,Kapan,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",39.20755,46.40576,P,PPLA,AM,None,08,None,None,None,33160,NaN,774,Asia/Yerevan,2023-10-23
1,95,174895,Goris,Goris,"Geryusy,Goris,Горис,Գորիս",39.51111,46.34168,P,PPL,AM,None,08,None,None,None,20379,NaN,1351,Asia/Yerevan,2018-03-12
2,96,174972,Hats’avan,Hats'avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",39.46405,45.97047,P,PPL,AM,None,08,None,None,None,15208,NaN,1761,Asia/Yerevan,2020-06-10


In [23]:
# Исследуем датасет
info_df(df_cities1500)

Размер таблиы: 20 столбцов и 1711 строк
Общее количество пропусков - 8297
alternatenames      19
cc2               1711
admin2 code       1519
admin3 code       1654
admin4 code       1711
elevation         1683
dtype: int64

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711 entries, 0 to 1710
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              1711 non-null   int64  
 1   geonameid          1711 non-null   int64  
 2   name               1711 non-null   object 
 3   asciiname          1711 non-null   object 
 4   alternatenames     1692 non-null   object 
 5   latitude           1711 non-null   float64
 6   longitude          1711 non-null   float64
 7   feature class      1711 non-null   object 
 8   feature code       1711 non-null   object 
 9   country code       1711 non-null   object 
 10  cc2                0 non-null      object 
 11  admin1

,index,geonameid,latitude,longitude,population,elevation,dem
count,1711.00,1711.00,1711.00,1711.00,1711.00,28.00,1711.00
mean,19831.46,1001068.44,49.74,48.02,119945.21,419.71,288.93
std,3908.43,1352497.94,7.83,23.54,505508.17,361.52,368.11
min,94.00,174875.00,36.08,18.98,15019.00,10.00,-25.00
25%,20133.50,493582.50,41.66,34.32,22501.50,111.75,81.50
50%,20561.00,559654.00,52.10,39.78,39214.00,345.00,152.00
75%,20988.50,827692.00,55.77,54.66,84136.00,747.50,293.00
max,22380.00,12041452.00,69.49,177.51,14804116.00,1276.00,2087.00


In [24]:
# Определим список исследуемых geonameid
geonameid_list = df_cities1500['geonameid'].unique()

# Сократим число столбцов
df_cities1500 = df_cities1500.loc[:, ['geonameid', 'name', 'alternatenames', 'country code', 'admin1 code', 'population']].copy()

# Удалим пропуски 
df_cities1500 = df_cities1500.dropna(subset=['alternatenames'])

In [25]:
# Добавим столбец со словарем аоьтернативных названий
df_cities1500['dict_text'] = df_cities1500.apply(lambda x: alternatenames(x['alternatenames'], x['geonameid']), axis=1)

In [26]:
df_cities1500.head(3)

,geonameid,name,alternatenames,country code,admin1 code,population,dict_text
0,174875,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",AM,08,33160,"{'Ghap'an': 174875, 'Ghapan': 174875, 'Ghap’an..."
1,174895,Goris,"Geryusy,Goris,Горис,Գորիս",AM,08,20379,"{'Geryusy': 174895, 'Goris': 174895, 'Горис': ..."
2,174972,Hats’avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",AM,08,15208,"{'Acavan': 174972, 'Atsavan': 174972, 'Hats'av..."


In [27]:
# Объединим словари
big_list = list(df_cities1500['dict_text'])
big_diht = {}
for i in big_list:
    big_diht.update(i)

In [28]:
# Создадим новый дата-фрейм
df_altern = pd.DataFrame(list(big_diht.items()))
df_altern.columns = ['alternatenames', 'geonameid']

In [29]:
df_altern.head()

,alternatenames,geonameid
0,Ghap'an,174875
1,Ghapan,174875
2,Ghap’an,174875
3,Kafan,174875
4,Kafin,174875


In [30]:
# Исследуем датасет
info_df(df_altern)

Размер таблиы: 2 столбцов и 24178 строк
Общее количество пропусков - 0

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24178 entries, 0 to 24177
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   alternatenames  24178 non-null  object
 1   geonameid       24178 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 377.9+ KB
None



,geonameid
count,24178.00
mean,850811.38
std,760143.60
min,174875.00
25%,498817.00
50%,561667.00
75%,792078.00
max,12041452.00


**Вывод**

Предоставленный датасет с характеристиками городов излишен по объему. В связи с этим, в целях ускорения решения поставленной задачи, в полученном датасете оставлены только города, входящие в список исследуемых стран.

Для обучающего датасета необходимо учесть все варианты написания названия города. Для этого список в столбце альтернативных названий преобразован в словарь, из которого сформирован новый обучающий датасет.


### Названия административных подразделений на английском языке<a id='2.3'></a>

In [31]:
# Названия столбцов
admin1Codes_names=['code', 'name', 'name ascii', 'geonameid']

In [32]:
# Чтение файла
try:
    df_admin1CodesASCII = pd.read_csv(patch + 'admin1CodesASCII.txt', sep='\t', 
                                      lineterminator='\n', header=None, names=admin1Codes_names)
except FileNotFoundError:
    df_admin1CodesASCII = pd.read_csv(patch_1 + 'admin1CodesASCII.txt', sep='\t', 
                                      lineterminator='\n', header=None, names=admin1Codes_names)

In [33]:
df_admin1CodesASCII.head(3)

,code,name,name ascii,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131


In [34]:
# Проверка кода региона
df_admin1CodesASCII[df_admin1CodesASCII['code']=='RU.91']

,code,name,name ascii,geonameid
2820,RU.91,Krasnoyarsk Krai,Krasnoyarsk Krai,1502020


In [35]:
# Исследуем датасет
info_df(df_admin1CodesASCII)

Размер таблиы: 4 столбцов и 3880 строк
Общее количество пропусков - 0

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3880 entries, 0 to 3879
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   code        3880 non-null   object
 1   name        3880 non-null   object
 2   name ascii  3880 non-null   object
 3   geonameid   3880 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 121.4+ KB
None



,geonameid
count,3880.00
mean,3205812.09
std,2833536.93
min,50360.00
25%,934508.00
50%,2565342.50
75%,3680855.00
max,12510145.00


### Обучающий датасет с альтернативными именами и кодами языков<a id='2.4'></a>

In [36]:
# Названия столбцов
alternateNames_names=['alternateNameId', 'geonameid', 'isolanguage', 'alternate name',
                      'isPreferredName', 'isShortName', 'isColloquial', 'isHistoric',
                      'from', 'to']

In [37]:
# Чтение файла
try:
    df_alternateNamesV2 = pd.read_csv(patch + 'alternateNamesV2.txt', sep='\t', lineterminator='\n', 
                                      header=None, low_memory=False, names=alternateNames_names)
except FileNotFoundError:
    df_alternateNamesV2 = pd.read_csv(patch_1 + 'alternateNamesV2.txt', sep='\t', lineterminator='\n', 
                                      header=None, low_memory=False, names=alternateNames_names)

In [38]:
# Оставим только исследуемые страны
df_alternateNamesV2 = df_alternateNamesV2.query('geonameid in @geonameid_list').reset_index(drop=True)

In [39]:
# Создание таблицы на сервере
df_alternateNamesV2.to_sql('alternateNamesV2', con=engine)

753

In [40]:
query = """

SELECT * 
FROM public."alternateNamesV2"
LIMIT 5

"""

pd.read_sql_query(text(query), con=engine.connect())

,index,alternateNameId,geonameid,isolanguage,alternate name,isPreferredName,isShortName,isColloquial,isHistoric,from,to
0,0,135616,174875,None,Qafan,None,None,None,None,None,None
1,1,1925363,174875,es,Kapan,None,None,None,None,None,None
2,2,1925364,174875,en,Kapan,None,None,None,None,None,None
3,3,1925365,174875,de,Kapan,None,None,None,None,None,None
4,4,1925366,174875,fa,کاپان,None,None,None,None,None,None


In [41]:
# Выгрузка таблицы с сервера

query = """

SELECT * 
FROM public."alternateNamesV2"

"""

df_alternateNamesV2 = pd.read_sql_query(text(query), con=engine.connect())

In [42]:
df_alternateNamesV2.head(3)

,index,alternateNameId,geonameid,isolanguage,alternate name,isPreferredName,isShortName,isColloquial,isHistoric,from,to
0,0,135616,174875,None,Qafan,NaN,NaN,NaN,NaN,None,None
1,1,1925363,174875,es,Kapan,NaN,NaN,NaN,NaN,None,None
2,2,1925364,174875,en,Kapan,NaN,NaN,NaN,NaN,None,None


In [43]:
# Исследуем датасет
info_df(df_alternateNamesV2)

Размер таблиы: 11 столбцов и 29753 строк
Общее количество пропусков - 181080
isolanguage         4843
isPreferredName    28347
isShortName        29721
isColloquial       29726
isHistoric         29300
from               29537
to                 29606
dtype: int64

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29753 entries, 0 to 29752
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            29753 non-null  int64  
 1   alternateNameId  29753 non-null  int64  
 2   geonameid        29753 non-null  int64  
 3   isolanguage      24910 non-null  object 
 4   alternate name   29753 non-null  object 
 5   isPreferredName  1406 non-null   float64
 6   isShortName      32 non-null     float64
 7   isColloquial     27 non-null     float64
 8   isHistoric       453 non-null    float64
 9   from             216 non-null    object 
 10  to               147 non-null    obj

,index,alternateNameId,geonameid,isPreferredName,isShortName,isColloquial,isHistoric
count,29753.0,29753.00,29753.00,1406.0,32.0,27.0,453.0
mean,14876.0,6406562.62,845479.65,1.0,1.0,1.0,1.0
std,8589.1,4601673.12,729517.77,0.0,0.0,0.0,0.0
min,0.0,300.00,174875.00,1.0,1.0,1.0,1.0
25%,7438.0,1927090.00,498687.00,1.0,1.0,1.0,1.0
50%,14876.0,7183220.00,560756.00,1.0,1.0,1.0,1.0
75%,22314.0,9379115.00,792680.00,1.0,1.0,1.0,1.0
max,29752.0,17681976.00,12041452.00,1.0,1.0,1.0,1.0


**Список альтернативных названий**

Создадим обучающую выборку

In [44]:
df_altern.head(2)

,alternatenames,geonameid
0,Ghap'an,174875
1,Ghapan,174875


In [45]:
# Создадим вторую часть датафейма альтернативных названий
df_altern_alternateNamesV2 = df_alternateNamesV2.loc[:, ['geonameid', 'alternate name']].copy()
df_altern_alternateNamesV2 = df_altern_alternateNamesV2.drop_duplicates().reset_index(drop=True)
df_altern_alternateNamesV2.columns = ['geonameid', 'alternatenames']

In [46]:
df_altern_alternateNamesV2.tail(2)

,geonameid,alternatenames
21285,11238838,https://tr.wikipedia.org/wiki/Merkezefendi
21286,11238838,https://en.wikipedia.org/wiki/Merkezefendi


In [47]:
# Объединим датафреймы
df_altern_all = pd.concat([df_altern, df_altern_alternateNamesV2], keys=['alternatenames', 'geonameid']).reset_index(drop=True)

In [48]:
df_altern_all.head(2)

,alternatenames,geonameid
0,Ghap'an,174875
1,Ghapan,174875


In [49]:
df_altern_all.tail(2)

,alternatenames,geonameid
45463,https://tr.wikipedia.org/wiki/Merkezefendi,11238838
45464,https://en.wikipedia.org/wiki/Merkezefendi,11238838


In [50]:
# Удалим строки, содержащие ссылки
df_altern_all = df_altern_all[~df_altern_all.alternatenames.str.contains('https://')].copy()

In [51]:
df_altern_all.tail(2)

,alternatenames,geonameid
45459,TRYLI,7628419
45462,Muratpaşa,8074174


In [52]:
# Наименования приведем к нижниму регистру и очистим от лишних символов
df_altern_all['alternatename'] = df_altern_all['alternatenames'].apply(clear_text)
df_altern_all.drop('alternatenames', axis=1, inplace=True)
df_altern_all = df_altern_all.drop_duplicates().reset_index(drop=True)

In [53]:
# Создадим столбец с транслитированными русскими названиями
df_altern_all['alternatename_1'] = df_altern_all['alternatename'].apply(translit)
df_altern_all['alternatename_en'] = df_altern_all['alternatename_1'].apply(clear_text)
df_altern_all.drop('alternatename_1', axis=1, inplace=True)

In [54]:
df_altern_all.head(2)

,geonameid,alternatename,alternatename_en
0,174875,ghap an,ghap an
1,174875,ghapan,ghapan


In [55]:
df_altern_all[df_altern_all['alternatename']=='петропавловськ-камчатський']

,geonameid,alternatename,alternatename_en
15791,2122104,петропавловськ-камчатський,petropavlovs k-kamchats kiy


In [56]:
# Исследуем датасет
info_df(df_altern_all)

Размер таблиы: 3 столбцов и 21232 строк
Общее количество пропусков - 0

Количество явных дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21232 entries, 0 to 21231
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   geonameid         21232 non-null  int64 
 1   alternatename     21232 non-null  object
 2   alternatename_en  21232 non-null  object
dtypes: int64(1), object(2)
memory usage: 497.8+ KB
None



,geonameid
count,21232.00
mean,852851.82
std,785273.78
min,174875.00
25%,498817.00
50%,559317.00
75%,789923.00
max,12041452.00


In [57]:
# Создание обучающего датафрейма на сервере
df_altern_all.to_sql('altern_all', con=engine)

232

In [58]:
# Выгрузка таблицы с сервера

query = """

SELECT * 
FROM altern_all

"""

df_altern_all = pd.read_sql_query(text(query), con=engine.connect())

**Вывод**

Предоставленный датасет с альтернативными именами, также излишен по объему. В связи с этим, как и с датасетом о городах, в целях ускорения решения поставленной задачи, в датасете альтернативных имен оставлены только города, входящие в список исследуемых стран.

Все альтернативные названия добавлены в обучающий датасет. Из обучающего датасета удалены ссылки, проведена очистка текста и создан столбец с транслитированным наименованием.

Обучающий датасет загружен на сервер для дальнейшей работы с ним


## Обучение<a id='3'></a>

**Рзделение признаков**

In [59]:
# Обучающий корпус
corpus = pd.Series(df_altern_all['alternatename'])
corpus_en = pd.Series(df_altern_all['alternatename_en'])
# Целевой показатель
answ = pd.Series(df_altern_all['geonameid'])

### Евклидово расстояние для кириллицы<a id='3.1'></a>

In [60]:
# создаем мешок слов
vectorizer_e = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
bow_e = vectorizer_e.fit_transform(corpus)
vectorizer_e.get_feature_names_out()

array([' -', ' 0', ' 1', ..., 'ёп', 'ёр', 'ёх'], dtype=object)

In [61]:
bow_e.shape

(21232, 1711)

In [62]:
# Запрос
text = 'ekaterinbyrg'

In [63]:
# Преобразуем входную строку в вектор 
questi = pd.Series(clear_text(text))
query_vec = vectorizer_e.transform(questi)

In [64]:
# Число строк вывода
n_strings = 5

# вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
distances = euclidean_distances(query_vec, bow_e)

# получаем индекс наиболее близкой строки
top_indices = distances.argsort()[0][:n_strings]

for tops in range(n_strings):
    # Ответ
    answ_text = answ[int(top_indices[tops])]
    print(answ_text)

    # выводим наиболее близкую строку
    top_strings = [corpus[i] for i in top_indices]
    print(top_strings[tops])

    print(df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False))
    print()

1486209
ekaterinburg
Yekaterinburg

1486209
ekaterinbourg
Yekaterinburg

1486209
jekaterinburg
Yekaterinburg

1486209
ekaterinburgo
Yekaterinburg

1486209
yekaterinburg
Yekaterinburg



Создадим функцию для рассчета Евклидова расстояниа без транслитирования

In [65]:
# Функция расчета Евклидова расстояния без транслитирования

def euclid_rus(text, num=1):
    
    # Преобразуем входную строку в вектор 
    questi = pd.Series(clear_text(text))
    query_vec = vectorizer_e.transform(questi)
    
    # вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
    distances = euclidean_distances(query_vec, bow_e)

    # получаем индекс наиболее близкой строки
    top_indices = distances.argsort()[0][:num]
    
    # Список вывода 
    top_strings_list = []

    for tops in range(num):
        # Ответ
        answ_text = answ[int(top_indices[tops])]
        answ_str = df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False)
        top_strings_list.append(answ_str)
        
    return top_strings_list


In [66]:
euclid_rus('Petersburg', 5)

['Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg']

### Косинусное расстояние<a id='3.2'></a>

In [67]:
# создаем мешок слов
vectorizer_c = CountVectorizer(analyzer='char_wb', ngram_range=(1, 3))
bow_c = vectorizer_c.fit_transform(corpus)
vectorizer_c.get_feature_names_out()

array([' ', ' -', ' - ', ..., 'ёры', 'ёх', 'ёхг'], dtype=object)

In [68]:
# Преобразуем входную строку в вектор 
questi = clear_text(text)
query_vec = vectorizer_c.transform([questi]).toarray()[0]

In [69]:
# Расчет косинусного расстояния
cos_distances = {}
for word in corpus:
    word_vect = vectorizer_c.transform([word]).toarray()[0]
    distances = 1-distance.cosine(word_vect, query_vec)
    cos_distances[word] = distances


C:\Users\User\miniconda3\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [70]:
sorted_cos_distances = sorted(cos_distances.items(), key=lambda x: x[1])
print(sorted_cos_distances[:5])

[('петропавловськ-камчатський', 0.05738343500421794), ('петропавловск-камчатский', 0.0606966995739181), ('петропавловск-камчатски', 0.061618727544822005), ('орехово-борисово', 0.06258409275407506), ('очаково-матвеевское', 0.06359630007488937)]


### Евклидово расстояние с переводом в латиницу<a id='3.3'></a>

In [71]:
# создаем мешок слов
vectorizer_en = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
bow_en = vectorizer_en.fit_transform(corpus_en)
vectorizer_en.get_feature_names_out()[:30]

array([' -', ' 0', ' 1', ' 2', ' 3', ' 6', ' a', ' b', ' c', ' d', ' e',
       ' f', ' g', ' h', ' i', ' j', ' k', ' l', ' m', ' n', ' o', ' p',
       ' q', ' r', ' s', ' t', ' u', ' v', ' w', ' x'], dtype=object)

In [72]:
bow_en.shape

(21232, 889)

In [73]:
# Запрос
text = 'СПб' 

In [74]:
# Преобразуем входную строку в вектор 
questi = pd.Series(clear_text(translit(text)))
query_vec = vectorizer_en.transform(questi)

In [75]:
# Число строк вывода
n_strings = 3

# вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
distances = euclidean_distances(query_vec, bow_en)

# получаем индекс наиболее близкой строки
top_indices = distances.argsort()[0][:n_strings]

for tops in range(n_strings):
    # Ответ
    answ_text = answ[int(top_indices[tops])]
    print(answ_text)

    # выводим наиболее близкую строку
    top_strings = [corpus[i] for i in top_indices]
    print(top_strings[tops])

    print(df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False))
    print()

498817
spb
Saint Petersburg

498817
спб
Saint Petersburg

1505429

Iskitim



Создадим функцию для рассчета Евклидова расстояниа с транслитированием

In [76]:
# Функция расчета Евклидова расстояния с переводом в латиннницу

def euclid_en(text, num=1):
    
    # Преобразуем входную строку в вектор 
    questi = pd.Series(clear_text(translit(text)))
    query_vec = vectorizer_en.transform(questi)
    
    # вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
    distances = euclidean_distances(query_vec, bow_en)

    # получаем индекс наиболее близкой строки
    top_indices = distances.argsort()[0][:num]
        
    # Список вывода 
    top_strings_list = []

    for tops in range(num):
        # Ответ
        answ_text = answ[int(top_indices[tops])]
        answ_str = df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False)
        top_strings_list.append(answ_str)
        
    return top_strings_list


In [77]:
euclid_en('Petersburg', 5)

['Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg',
 'Saint Petersburg']

### Расстояние Левенштейна для русского текста <a id='3.4'></a>

In [78]:
# Запрос
text = 'Масква'

In [79]:
# Число строк вывода
n_strings = 5

# вычисляем расстояние Левенштейна между новой строкой и всеми строками корпусе
dist_lev_list = []
for word in corpus:
    dist_lev = fuzz.token_set_ratio(text, word)
    dist_lev_list.append(dist_lev)

# Формируем датафрейм с ответаи 
df_dist_lev = pd.DataFrame({
    'geonameid': list(answ),
    'alternatename': list(corpus),
    'dist_lev': dist_lev_list
})

# Определим наиболее точные совпадения
df_dist_lev = df_dist_lev.sort_values(by='dist_lev', ascending=False)
tops_list = list(df_dist_lev['dist_lev'].nlargest(n=n_strings).index)

for tops in tops_list:
    # Ответ
    answ_text = df_dist_lev.loc[tops, 'geonameid']
    print('geonameid', answ_text)

    # Значение
    answ_num = df_dist_lev.loc[tops, 'dist_lev']
    print('Значение',  answ_num)
    
    
    # выводим наиболее близкую строку
    top_strings = df_dist_lev.loc[tops, 'alternatename']
    print(top_strings)

    # Значение geoname
    print(df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False))
    print()

geonameid 524901
Значение 100
масква
Moscow

geonameid 524901
Значение 83
маскав
Moscow

geonameid 524901
Значение 83
москва
Moscow

geonameid 2056881
Значение 77
маркова
Markova

geonameid 524901
Значение 77
москъва
Moscow



Создадим функцию для рассчета расстояниа Левенштейна без транслитирования

In [80]:
# Функция расчета расстояния Левенштейна

def levenshteyn_dist_rus(text, num=1):
    
    # вычисляем расстояние Левенштейна между новой строкой и всеми строками корпусе
    dist_lev_list = []
    for word in corpus:
        dist_lev = fuzz.token_set_ratio(text, word)
        dist_lev_list.append(dist_lev)

    # Формируем датафрейм с ответаи    
    df_dist_lev = pd.DataFrame({
        'geonameid': list(answ),
        'alternatename': list(corpus),
        'dist_lev': dist_lev_list
    })

    # Определим наиболее точные совпадения
    df_dist_lev = df_dist_lev.sort_values(by='dist_lev', ascending=False)
    tops_list = list(df_dist_lev['dist_lev'].nlargest(n=num).index)

    # Список вывода 
    top_strings_list = []
    
    for tops in tops_list:
        
        # Ответ
        answ_text = df_dist_lev.loc[tops, 'geonameid']

        # Значение geoname
        answ_str = df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False)
        top_strings_list.append(answ_str)
    
    return top_strings_list

In [81]:
levenshteyn_dist_rus('Масква', 5)

['Moscow', 'Moscow', 'Moscow', 'Markova', 'Moscow']

### Расстояние Левенштейна с переводом в латиницу<a id='3.5'></a>

In [82]:
# Число строк вывода
n_strings = 5

# вычисляем расстояние Левенштейна между новой строкой и всеми строками корпусе
dist_lev_en_list = []
for word in corpus_en:
    dist_lev = fuzz.token_set_ratio(translit(text), word)
    dist_lev_en_list.append(dist_lev)

# Формируем датафрейм с ответаи 
df_dist_lev_en = pd.DataFrame({
    'geonameid': list(answ),
    'alternatename': list(corpus),
    'dist_lev': dist_lev_en_list
})

# Определим наиболее точные совпадения
df_dist_lev_en = df_dist_lev.sort_values(by='dist_lev', ascending=False)
tops_list_en = list(df_dist_lev_en['dist_lev'].nlargest(n=n_strings).index)

for tops in tops_list_en:
    # Ответ
    answ_text = df_dist_lev_en.loc[tops, 'geonameid']
    print('geonameid', answ_text)

    # Значение
    answ_num = df_dist_lev_en.loc[tops, 'dist_lev']
    print('Значение',  answ_num)
    
    
    # выводим наиболее близкую строку
    top_strings = df_dist_lev_en.loc[tops, 'alternatename']
    print(top_strings)

    # Значение geoname
    print(df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False))
    print()

geonameid 524901
Значение 100
масква
Moscow

geonameid 524901
Значение 83
москва
Moscow

geonameid 524901
Значение 83
маскав
Moscow

geonameid 2056881
Значение 77
маркова
Markova

geonameid 524901
Значение 77
москъва
Moscow



Создадим функцию для рассчета расстояниа Левенштейна с транслитированием

In [83]:
# Функция расчета расстояния Левенштейна с переводом в латиннницу

def levenshteyn_dist_en(text, num=1):
    
    # вычисляем расстояние Левенштейна между новой строкой и всеми строками корпусе
    dist_lev_en_list = []
    for word in corpus_en:
        dist_lev = fuzz.token_set_ratio(translit(text), word)
        dist_lev_en_list.append(dist_lev)
        
    # Формируем датафрейм с ответаи    
    df_dist_lev_en = pd.DataFrame({
        'geonameid': list(answ),
        'alternatename': list(corpus),
        'dist_lev': dist_lev_en_list
    })

    # Определим наиболее точные совпадения
    df_dist_lev_en = df_dist_lev_en.sort_values(by='dist_lev', ascending=False)
    tops_list_en = list(df_dist_lev_en['dist_lev'].nlargest(n=num).index)

    # Список вывода 
    top_strings_list = []
    
    for tops in tops_list_en:
        
        # Ответ
        answ_text = df_dist_lev_en.loc[tops, 'geonameid']

        # Значение geoname
        answ_str = df_cities1500.loc[df_cities1500['geonameid']==answ_text, 'name'].to_string(index=False)
        top_strings_list.append(answ_str)
    
    return top_strings_list

In [84]:
levenshteyn_dist_en('Масква', 3)

['Moscow', 'Moscow', 'Moscow']

**Вывод**

В рамках обучения модели были исследованы различные методы. В рамках достижения баланса между скоростью работы модели и ее точностью, акцент сделан на классических моделях – евклидово расстояние, косинусное расстояние и расстояние Левинсона. Для каждого метода было также применено транслитирование.

При проведении тестов косинусное расстояние показало низкое качество прогноза, в результате чего данный метод развивать было не целесообразно.

Т.о., к проверке приняты два метода (евклидово расстояние и расстояние Левинсона) как с транслитированием, так и без него.


## Проверка точности модели<a id='4'></a>

In [85]:
# Чтение тестового файла
df_test.head(3)

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan


**Формирование ответов для всех способов оценки**

In [86]:
%time
df_test['euclid_rus'] = df_test['query'].apply(euclid_rus).apply(lambda x: ''.join(x))
df_test['euclid_en'] = df_test['query'].apply(euclid_en).apply(lambda x: ''.join(x))
df_test['levenshteyn_rus'] = df_test['query'].apply(levenshteyn_dist_rus).apply(lambda x: ''.join(x))
df_test['levenshteyn_en'] = df_test['query'].apply(levenshteyn_dist_en).apply(lambda x: ''.join(x))

CPU times: total: 0 ns
Wall time: 0 ns


In [87]:
df_test.head()

,query,name,region,country,euclid_rus,euclid_en,levenshteyn_rus,levenshteyn_en
0,Смоленск,Smolensk,Smolensk Oblast,Russia,Smolensk,Smolensk,Smolensk,Smolensk
1,Кемерово,Kemerovo,Kuzbass,Russia,Kemerovo,Kemerovo,Kemerovo,Kemerovo
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan,Bishkek,Bishkek,Bishkek,Bishkek
3,Москва,Moscow,Moscow,Russia,Moscow,Moscow,Moscow,Moscow
4,Алматы,Almaty,Almaty,Kazakhstan,Almaty,Almaty,Almaty,Almaty


In [88]:
y_true = df_test['name']
y_pred_ev_rus = df_test['euclid_rus']
y_pred_ev_en = df_test['euclid_en']
y_pred_lev_rus = df_test['levenshteyn_rus']
y_pred_lev_en = df_test['levenshteyn_en']

accuracy_euclid_rus = accuracy_score(y_true, y_pred_ev_rus)
accuracy_euclid_en = accuracy_score(y_true, y_pred_ev_en)
accuracy_levenshteyn_rus = accuracy_score(y_true, y_pred_lev_rus)
accuracy_levenshteyn_en = accuracy_score(y_true, y_pred_lev_en)

print(f'Для Евклидова расстояния на русском языке accuracy составила {accuracy_euclid_rus:.4f}')
print(f'Для Евклидова расстояния с переводом на английский язык accuracy составила {accuracy_euclid_en:.4f}')
print(f'Для расстояния Левенштейна на русском языке accuracy составила {accuracy_levenshteyn_rus:.4f}')
print(f'Для расстояния Левенштейна с переводом на английский язык accuracy составила {accuracy_levenshteyn_en:.4f}')

Для Евклидова расстояния на русском языке accuracy составила 0.9294
Для Евклидова расстояния с переводом на английский язык accuracy составила 0.9206
Для расстояния Левенштейна на русском языке accuracy составила 0.9088
Для расстояния Левенштейна с переводом на английский язык accuracy составила 0.9029


In [89]:
# Таблица ошибок
df_test.loc[df_test['name']!=df_test['euclid_en']]

,query,name,region,country,euclid_rus,euclid_en,levenshteyn_rus,levenshteyn_en
10,Минск,Minsk City,Minsk City,Belarus,Minsk,Minsk,Minsk,Minsk
15,Екб,Yekaterinburg,Sverdlovsk Oblast,Russia,Serdobsk,Ekibastuz,Oral,Ekibastuz
17,Н.Новгород,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,Velikiy Novgorod,Velikiy Novgorod,Velikiy Novgorod,Velikiy Novgorod
31,Остана,Astana,Astana,Kazakhstan,Kostanay,Kostanay,Kostanay,Kostanay
44,Островцы,Ostrovtsy,Moscow Oblast,Russia,Ostrov,Ostrov,Ostrov,Ostrov
63,Ираславль,Yaroslavl,Yaroslavl Oblast,Russia,Roslavl’,Roslavl’,Roslavl’,Yaroslavl
67,Влодевасток,Vladivostok,Primorye,Russia,Vladivostok,Volgodonsk,Vladivostok,Vladivostok
75,Аксай,Aksay,Rostov,Russia,Aqsay,Aqsay,Aksay,Aksay
85,Каленинград,Kaliningrad,Kaliningrad Oblast,Russia,Kaliningrad,Korolev,Kaliningrad,Kaliningrad
86,Калининград,Kaliningrad,Kaliningrad Oblast,Russia,Kaliningrad,Korolev,Kaliningrad,Kaliningrad


**Вывод**

Скорость расчета расстояния Левенштейна крайне низкая, при сравнительно низкой точности (`accuracy` не превысила `0,9`).

Самая высокая точность достигается на Евклидовом расстоянии без применения транслитирования (`accuracy` составила `0.9235`). 

Однако, допуская возможность ввода названий в латинице, выбран метод с тарнслитированием в английский язык, на котором `accuracy` достигает значения `0.9206`.

Таблица ошибок показала класс ошибок, когда пользователь вводит вместо города название страны. Для обработки такого рода ошибок возможно расширение функционала алгоритма в будущем.


## Формирование словаря ответов<a id='5'></a>

**Описание**

Для организации работы на уже обученной модели машинного обучения предложен алгоритм формирования словаря ответов. 

Для запуска работы алгоритма необходимо векторизировать обучающий корпус наименований и запустить функцию. 

При вводе наименования можно вводить не только наименование, но и количество строк вывода (по умолчанию выводится одна строка).

В результате работы функции выводится список словарей с указанием `geonameid`, наименования, `города` / `региона` / `страны` и значения `косинусного расстояния` между введенным и полученным значением.


**Расчет косинусоного расстояния**

In [90]:
# Функция расчета косинусного расстояния
def cos_dist(questi, word):
    query_vec = vectorizer_c.transform([clear_text(translit(questi))]).toarray()[0]
    word_vect = vectorizer_c.transform([clear_text(word)]).toarray()[0]
    distances = 1-distance.cosine(word_vect, query_vec)
    return distances

**База для формирования ответов**

In [91]:
# создаем мешок слов
vectorizer_en = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
bow_en = vectorizer_en.fit_transform(corpus_en)

In [92]:
bow_en.shape

(21232, 889)

In [93]:
# Запрос
text = 'Екатеринбург' 

**Формирование ответов**

In [94]:
# Преобразуем входную строку в вектор 
questi = pd.Series(clear_text(translit(text)))
query_vec = vectorizer_en.transform(questi)

In [95]:
# Создание словаря ответов
# Число строк вывода
n_strings = 5

# вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
distances = euclidean_distances(query_vec, bow_en)

# получаем индекс наиболее близкой строки
top_indices = distances.argsort()[0][:n_strings]

geonameid_list = []
name_list = []
region_list =[]
country_list = []
cosine_list =[]

for tops in range(n_strings):
    # Ответ geonameid
    answ_geonameid = answ[int(top_indices[tops])]
    geonameid_list.append(answ_geonameid)
    
    # Ответ name
    answ_name = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'name'].to_string(index=False)
    name_list.append(answ_name)
    
    # Ответ регион
    cc = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'country code'].to_string(index=False)
    rc = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'admin1 code'].to_string(index=False)
    ar = cc +'.'+ rc
    answ_region = df_admin1CodesASCII.loc[df_admin1CodesASCII['code']==ar, 'name ascii'].to_string(index=False)
    region_list.append(answ_region)
    
    # Ответ страна
    answ_country = country_name[df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'country code'].
                                to_string(index=False)]
    country_list.append(answ_country)
    
    # Ответ cosine similarity
    cosine_similarity = cos_dist(text, corpus_en[top_indices[tops]])
    cosine_list.append(cosine_similarity)

df_answ = pd.DataFrame({
    'geonameid': geonameid_list,
    'name': name_list,
    'region': region_list,
    'country': country_list,
    'cosine': cosine_list
})
answ_dict = df_answ.to_dict(orient='records')

In [96]:
answ_dict

[{'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 1.0},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 1.0},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606}]

**Функция формирования ответов**

In [97]:
# Функция для формирования ответов

def answ_dict(text, num=1):

    # Преобразуем входную строку в вектор 
    questi = pd.Series(clear_text(translit(text)))
    query_vec = vectorizer_en.transform(questi)

    # вычисляем евклидово расстояние между новой строкой и всеми строками в мешке слов
    distances = euclidean_distances(query_vec, bow_en)

    # получаем индекс наиболее близкой строки
    top_indices = distances.argsort()[0][:num]

    # Создаем списки данных
    geonameid_list = []
    name_list = []
    region_list =[]
    country_list = []
    cosine_list =[]

    for tops in range(num):
        # Ответ geonameid
        answ_geonameid = answ[int(top_indices[tops])]
        geonameid_list.append(answ_geonameid)

        # Ответ name
        answ_name = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'name'].to_string(index=False)
        name_list.append(answ_name)

        # Ответ регион
        cc = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'country code'].to_string(index=False)
        rc = df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'admin1 code'].to_string(index=False)
        ar = cc +'.'+ rc
        answ_region = df_admin1CodesASCII.loc[df_admin1CodesASCII['code']==ar, 'name ascii'].to_string(index=False)
        region_list.append(answ_region)

        # Ответ страна
        answ_country = country_name[df_cities1500.loc[df_cities1500['geonameid']==answ_geonameid, 'country code'].
                                    to_string(index=False)]
        country_list.append(answ_country)

        # Ответ cosine similarity
        cosine_similarity = cos_dist(text, corpus_en[top_indices[tops]])
        cosine_list.append(cosine_similarity)

    df_answ = pd.DataFrame({
        'geonameid': geonameid_list,
        'name': name_list,
        'region': region_list,
        'country': country_list,
        'cosine': cosine_list
    })
    return df_answ.to_dict(orient='records')

In [98]:
answ_dict('Екатеринбург', 5)

[{'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 1.0},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 1.0},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606},
 {'geonameid': 1486209,
  'name': 'Yekaterinburg',
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'cosine': 0.9252126882606606}]